In [ ]:
pip install librosa

Скачиваем звуковые файлы из дирректории, с которыми далее будем работать 

In [ ]:
import numpy as np
import pandas as pd
import librosa
import matplotlib.pyplot as plt
import IPython.display as ipd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score  
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

In [ ]:
data1 = "Стук двигателя.wav"
x1 , sr_x1 = librosa.load(data1)

In [ ]:
type(sr_x1), type(x1)

Удостоверимся, что файл действительно загружен и мы можем его прослушать

In [ ]:
ipd.Audio(data1) 

Выведем спектрограмму обрабатываемого файла и проанализируем его

In [ ]:
X = librosa.stft(x1)
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=sr_x1, x_axis='time', y_axis='hz')
plt.colorbar()
plt.show()

In [ ]:
data2 = "Свист приводного ремня.wav"
x2 , sr_x2 = librosa.load(data2)

In [ ]:
ipd.Audio(data2)

In [ ]:
X = librosa.stft(x2)
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=sr_x2, x_axis='time', y_axis='hz')
plt.colorbar()
plt.show()

In [ ]:
data3 = "Неисправность выхлопной системы.mp3"
x3 , sr_x3 = librosa.load(data3)

In [ ]:
ipd.Audio(data3)

In [ ]:
X = librosa.stft(x3)
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=sr_x3, x_axis='time', y_axis='hz')
plt.colorbar()
plt.show()

In [ ]:
print(x1, x2, x3)

In [ ]:
y1 = np.full(len(x1), 0)  # Первый класс - Стук двигателя
y2 = np.full(len(x2), 1)  # Второй класс - Свист приводного ремня
y3 = np.full(len(x3), 2)  # Третий класс - Неисправность выхлопной системы

In [ ]:
data = np.concatenate((x1, x2, x3)) # Объединение массивов в одну таблицу
labels = np.concatenate((y1, y2, y3))

In [ ]:
df = pd.DataFrame({'data': data, 'labels': labels}) # Создание DataFrame из данных

In [ ]:
df

При анализе каждой из используемых далее моделей машинного обучения понадобятся метрики и график матрицы ошибок для каждой из них. Чтобы не писать аналогичный код много раз, создадим функцию evaluate_model(), которая будет выполнять эти действия, в дальнейшем будем её вызывать.

In [ ]:
def evaluate_model(model_name, classifier, X_test, y_test):
    
    y_pred = classifier.predict(X_test.tolist()) # Предсказание меток для тестовых данных
    cm = confusion_matrix(y_test.tolist(), y_pred) # Матрица ошибок

    # Создание графика матрицы ошибок
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(f'Confusion Matrix - {model_name}')
    plt.colorbar()
    tick_marks = np.arange(len(np.unique(y_test.tolist())))
    plt.xticks(tick_marks, np.unique(y_test.tolist()))
    plt.yticks(tick_marks, np.unique(y_test.tolist()))
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

    # Подсчёт метрик нашей модели
    classification_rep = classification_report(y_test.tolist(), y_pred) 
    accuracy = accuracy_score(y_test.tolist(), y_pred)
    precision = precision_score(y_test.tolist(), y_pred, average='weighted', zero_division=1)
    recall = recall_score(y_test.tolist(), y_pred, average='weighted', zero_division=1)
    f1 = f1_score(y_test.tolist(), y_pred, average='weighted', zero_division=1)
    
    # Вывод самих метрик на экран
    print(f'Classification Report - {model_name}:\n{classification_rep}')
    print(f'Accuracy - {model_name}: {accuracy}')
    print(f'Precision - {model_name}: {precision}')
    print(f'Recall - {model_name}: {recall}')
    print(f'F1 Score - {model_name}: {f1}')


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['data'], df['labels'], test_size=0.2, random_state=42) # Разделение данных на обучающую и тестовую выборки

Сейчас массивы одномерны. Для дальнейшего использования понадобится их преобразовывать в двумерный вид. Чтобы не делать этого на каждом этапе вычислений, сделаем это один раз ещё до начала применения всех моделей. 

In [ ]:
X_train = X_train.values.reshape(-1, 1)
X_test = X_test.values.reshape(-1, 1)

#  Модель 1: Decision Tree Classifier

In [ ]:
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train, y_train)

In [ ]:
evaluate_model("Decision Tree", dt_classifier, X_test, y_test)

# Модель 2: K-Nearest Neighbors Classifier

In [ ]:
knn_classifier = KNeighborsClassifier()
knn_classifier.fit(X_train.tolist(), y_train.tolist())

In [ ]:
evaluate_model("K-Nearest Neighbors", knn_classifier, X_test, y_test)

# Модель 3: Support Vector Classifier

In [ ]:
svc_classifier = LinearSVC()
svc_classifier.fit(X_train.tolist(), y_train.tolist())

In [ ]:
evaluate_model("Support Vector", svc_classifier, X_test, y_test)

# Модель 4: Random Forest Classifier

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=50, n_jobs=-1)
rf_classifier.fit(X_train.tolist(), y_train.tolist())

In [ ]:
evaluate_model("Random Forest Classifier", rf_classifier, X_test, y_test)

# Модель 5: Gaussian Naive Bayes Classifier

In [ ]:
nb_classifier = GaussianNB()
nb_classifier.fit(X_train.tolist(), y_train.tolist())

In [ ]:
evaluate_model("Gaussian Naive Bayes", nb_classifier, X_test, y_test)

# Модель 6: Logistic Regression Classifier

In [ ]:
lr_classifier = LogisticRegression()
lr_classifier.fit(X_train.tolist(), y_train.tolist())

In [ ]:
evaluate_model("Logistic Regression", lr_classifier, X_test, y_test)

# Модель 7: Multi-layer Perceptron Classifier

In [ ]:
mlp_classifier = MLPClassifier(hidden_layer_sizes=(50,), activation='relu', alpha=0.001, solver='adam', batch_size=100)
mlp_classifier.fit(X_train.tolist(), y_train.tolist())

In [ ]:
evaluate_model("Multi-layer Perceptron", mlp_classifier, X_test, y_test)

Лучше всего себя показали: модель 7 - Multi-layer Perceptron Classifier (Многослойный персептрон) и 5 - Gaussian Naive Bayes Classifier (Гауссовский наивный байесовский классификатор). Из-за сложности увеличения объёма датасета и ограниченной мощности ноутбука, на котором выполнялась работа, максимальная точность модели не превысила 64%, однако на более объёмном датасете этот показатель может существенно возрасти, но и в таком случае модель 7 по-прежнему будет наиболее эффективной среди остальных.  